In [ ]:
import os
import json
import re
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

dataset_folder = "/content/drive/MyDrive/mmml_project/outputs"

file_paths = {
    "llava": os.path.join(dataset_folder, "mini_gqa_with_llava.json"),
    "gpt": os.path.join(dataset_folder, "mini_gqa_with_gpt.json"),
    "smolvlm": os.path.join(dataset_folder, "mini_gqa_with_smolvlm.json"),
    "blip": os.path.join(dataset_folder, "mini_gqa_with_blip.json"),
}

datasets = {}
for model, path in file_paths.items():
    with open(path, "r") as f:
        datasets[model] = json.load(f)

Mounted at /content/drive


In [ ]:
def clean_llava_response(response):
    cleaned_response = re.sub(r"\[INST\].*?\n?.*?\[/INST\]", "", response, flags=re.DOTALL).strip()
    return cleaned_response

for record in datasets["llava"]:
    record["llava_response"] = clean_llava_response(record.get("llava_response", ""))

llava_cleaned_path = os.path.join(dataset_folder, "mini_gqa_with_llava_cleaned.json")
with open(llava_cleaned_path, "w") as f:
    json.dump(datasets["llava"], f, indent=2)

print(f"Cleaned LLaVA dataset saved to: {llava_cleaned_path}")

Cleaned LLaVA dataset saved to: /content/drive/MyDrive/mmml_project/outputs/mini_gqa_with_llava_cleaned.json


In [ ]:
def clean_smolvlm_response(response, question):
    response_cleaned = response.strip()
    question_cleaned = question.strip()

    if response_cleaned.lower().startswith(question_cleaned.lower()):
        response_cleaned = response_cleaned[len(question_cleaned):].strip()

    return response_cleaned

for record in datasets["smolvlm"]:
    question = record.get("question", "")
    record["smolvlm_response"] = clean_smolvlm_response(record.get("smolvlm_response", ""), question)

smolvlm_cleaned_path = os.path.join(dataset_folder, "mini_gqa_with_smolvlm_cleaned.json")
with open(smolvlm_cleaned_path, "w") as f:
    json.dump(datasets["smolvlm"], f, indent=2)

print(f"Fully Cleaned SmolVLM dataset saved to: {smolvlm_cleaned_path}")

Fully Cleaned SmolVLM dataset saved to: /content/drive/MyDrive/mmml_project/outputs/mini_gqa_with_smolvlm_cleaned.json


In [ ]:
import pandas as pd
datasets = {}
for model, path in file_paths.items():
    with open(path, "r") as f:
        dataset_list = json.load(f)
        datasets[model] = {entry["imageId"]: entry for entry in dataset_list}

def create_csv(strong_model, weak_model, output_filename):
    matched_data = []

    for image_id, strong_entry in datasets[strong_model].items():
        weak_entry = datasets[weak_model].get(image_id)

        if weak_entry and strong_entry["question"] == weak_entry["question"]:
            matched_data.append({
                "imageId": image_id,
                "image_file": strong_entry["image_file"],
                "question": strong_entry["question"],
                "answer": strong_entry["answer"],
                "full_answer": strong_entry["fullAnswer"],
                "strong_model_response": strong_entry.get(f"{strong_model}_response", ""),
                "weak_model_response": weak_entry.get(f"{weak_model}_response", ""),
            })

    df = pd.DataFrame(matched_data)
    csv_path = os.path.join(dataset_folder, output_filename)
    df.to_csv(csv_path, index=False)
    print(f"CSV saved: {csv_path}")

create_csv("blip", "llava", "blip_vs_llava.csv")

✅ CSV saved: /content/drive/MyDrive/mmml_project/outputs/blip_vs_llava.csv


In [ ]:
import json
import os

dataset_folder = "/content/drive/MyDrive/mmml_project/outputs"

file_paths = {
    "gpt": os.path.join(dataset_folder, "mini_gqa_with_gpt.json"),
    "smolvlm": os.path.join(dataset_folder, "mini_gqa_with_smolvlm.json"),
    "blip": os.path.join(dataset_folder, "mini_gqa_with_blip.json"),
    "llava": os.path.join(dataset_folder, "mini_gqa_with_llava.json"),
}

for model, path in file_paths.items():
    try:
        with open(path, "r") as f:
            json.load(f)
        print(f"{model} JSON is valid: {path}")
    except json.JSONDecodeError as e:
        print(f"Error in {model} JSON file: {path}")
        print(f"JSON Error: {e}")

gpt JSON is valid: /content/drive/MyDrive/mmml_project/outputs/mini_gqa_with_gpt.json
smolvlm JSON is valid: /content/drive/MyDrive/mmml_project/outputs/mini_gqa_with_smolvlm.json
blip JSON is valid: /content/drive/MyDrive/mmml_project/outputs/mini_gqa_with_blip.json
llava JSON is valid: /content/drive/MyDrive/mmml_project/outputs/mini_gqa_with_llava.json
